In [1]:
from sqlalchemy import create_engine, text
from urllib.parse import quote


In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Creating engine which connect to MySQL
# Add Mysql Database credentials
user = 'root' # user name
pw = 'bane2804' # password
db = 'new_project' # database

In [4]:
# creating engine to connect database
engine = create_engine(f"mysql+pymysql://{user}:%s@localhost/{db}" %quote(f'{pw}'))

In [5]:
import pandas as pd

In [6]:
# loading data from database
sql = text('select * from zomato')
df = pd.read_sql_query(sql, con = engine.connect())

In [7]:
df

,name,online_order,book_table,rating,votes,location,rest_type,dish_liked,cuisines,approx_cost,type
0,Jalsa,Yes,Yes,4.1,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet
1,Spice Elephant,Yes,No,4.1,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,Buffet
2,San Churro Cafe,Yes,No,3.8,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,Buffet
3,Addhuri Udupi Bhojana,No,No,3.7,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,Buffet
4,Grand Village,No,No,3.8,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,Buffet
...,...,...,...,...,...,...,...,...,...,...,...
19964,Orchard Fresh,No,No,3.4,5,Marathahalli,Dessert Parlor,NA,"Ice Cream, Desserts",400,Delivery
19965,Mast Biryani - Night Express,No,No,3.5,17,East Bangalore,Delivery,NA,"North Indian, Biryani, Fast Food",500,Delivery
19966,Utopia,No,No,2.8,59,Marathahalli,Casual Dining,NA,"North Indian, Chinese",550,Delivery
19967,Ovenstory Pizza,Yes,No,4.1,74,Marathahalli,Delivery,"Chocolava, Paneer Pizza",Pizza,800,Delivery


In [8]:
sql = text('SELECT name, votes, rating FROM zomato ORDER BY votes DESC LIMIT 5')
df = pd.read_sql_query(sql, con = engine.connect())

In [9]:
df

,name,votes,rating
0,Truffles,14726,4.7
1,Truffles,14723,4.7
2,Truffles,14717,4.7
3,Truffles,14710,4.7
4,Truffles,14704,4.7


In [10]:
sql = text('SELECT name, rating, location, type FROM zomato where location = "Banashankari" ORDER BY rating DESC LIMIT 5')
df = pd.read_sql_query(sql, con = engine.connect())
df

,name,rating,location,type
0,Taaza Thindi,4.7,Banashankari,Dine-out
1,Onesta,4.6,Banashankari,NA
2,Onesta,4.6,Banashankari,Cafes
3,Onesta,4.6,Banashankari,Dine-out
4,Onesta,4.6,Banashankari,Delivery


In [11]:
sql = text('SELECT name, rating FROM zomato where location = "Indiranagar" ORDER BY approx_cost DESC LIMIT 1')
df = pd.read_sql_query(sql, con = engine.connect())
df

,name,rating
0,Bathinda Dine,4.1


In [12]:
sql = text('SELECT online_order, SUM(votes) as total_votes FROM zomato GROUP BY online_order')
df = pd.read_sql_query(sql, con = engine.connect())
df

,online_order,total_votes
0,Yes,3447270.0
1,No,1018902.0


In [14]:
sql = text('SELECT  type,  COUNT(*) AS total_restaurants,  SUM(votes) AS total_votes,  AVG(rating) AS avg_rating FROM zomato GROUP BY type')
df1 = pd.read_sql_query(sql, con = engine.connect())
df1

,type,total_restaurants,total_votes,avg_rating
0,Buffet,125,104947.0,3.543200
1,Cafes,727,252130.0,3.610729
2,NA,1145,1043289.0,3.877205
3,Delivery,9775,1630744.0,3.417166
4,Desserts,1484,196474.0,3.574933
5,Dine-out,6631,1222622.0,3.383999
6,Drinks & nightlife,31,5434.0,3.322581
7,Pubs and bars,51,10532.0,3.423529


In [23]:
sql = """SELECT
    type,
    COUNT(*) AS num_restaurants,
    SUM(votes) AS total_votes,
    AVG(rating) AS avg_rating
FROM
    zomato
GROUP BY
    type
HAVING
    SUM(votes) < (SELECT MAX(total_votes) FROM (
                    SELECT SUM(votes) AS total_votes
                    FROM zomato
                    GROUP BY type
                  ) AS subquery)
ORDER BY
    total_votes DESC
LIMIT 10;
"""
df = pd.read_sql_query(sql, con = engine.connect())
df

,type,num_restaurants,total_votes,avg_rating
0,Dine-out,6631,1222622.0,3.383999
1,NA,1145,1043289.0,3.877205
2,Cafes,727,252130.0,3.610729
3,Desserts,1484,196474.0,3.574933
4,Buffet,125,104947.0,3.543200
5,Pubs and bars,51,10532.0,3.423529
6,Drinks & nightlife,31,5434.0,3.322581


In [28]:
sql = """SELECT
    name,
    dish_liked as most_liked_dish,
    rating,
    votes
FROM
    zomato
where 
    rest_type= "Delivery" and online_order = "Yes"
    and rating=
    (select max(rating) from zomato where rest_type= "Delivery" and online_order ="Yes")
ORDER BY
    votes DESC  
LIMIT 1
;
"""
df = pd.read_sql_query(sql, con = engine.connect())
df

,name,most_liked_dish,rating,votes
0,eat.fit,"Healthy Breakfast, Salads, Date Pudding, Chick...",4.6,1193


In [31]:
sql = """SELECT
    name,
    rating,
    votes,
    online_order
FROM
    zomato
where 
    rating >3 and votes >= 150
    and online_order = "No"
;
"""
df = pd.read_sql_query(sql, con = engine.connect())
df

,name,rating,votes,online_order
0,Grand Village,3.8,166,No
1,Caf-Eleven,4.0,424,No
2,Corner House Ice Cream,4.3,345,No
3,Roving Feast,4.0,1047,No
4,Peppy Peppers,4.2,244,No
...,...,...,...,...
983,Priyadarshini Veg,3.9,156,No
984,Hotel Janardhan,4.1,485,No
985,An Egg Story,4.1,707,No
986,Nandhana Palace,4.2,1459,No
